In [13]:
import json
import os
import requests

def read_chunks(chunk_dir):
    chunks = []
    for filename in os.listdir(chunk_dir):
        if filename.endswith(".json"):
            with open(os.path.join(chunk_dir, filename), 'r', encoding='utf-8') as f:
                chunk_data = json.load(f)
                chunks.append(chunk_data)
    return chunks

In [4]:
count_chunks = len(read_chunks('./chunks'))
print(read_chunks('./chunks')[1]['content'])

Клиент  - физическое лицо, достигшее возраста четырнадцать лет, осуществившее акцепт 
Соглашения в соответствии с его условиями.

При осуществлении Перевода Клиент может 
выступать в качестве Плательщика и Получателя.

Лицевой счет  - лицевой счет Клиента в Системе МОНЕТА.РУ , открытый на балансовом  
счете 40903 или  40914 , применяется в соответствии с Положением Банка России от 24 ноября 2022 
г.№ 809-П «О плане счетов бухгалтерского учета для кредитных организаций и порядке его 
применения» , не является банковским счетом Клиента , имеет уникальный номер в Системе 
МОНЕТА.РУ , может быть несколько счетов в  разных валютах .

Личный кабинет  - средство управления Кошельком.

Раздел на Сайте Оператора  в сети 
Интернет, содержащий данные Клиента, Лицевые счета и иную информацию, определяемую 
Оператором.

Доступ в Личный кабинет осуществляется по протоколу HTTPS, с предоставлением 
возможности настройки двухфакторной аутентификации с использованием протокола TOTP.

Нерезиденты  - физ

In [14]:
def llm_request(prompt: str, api_key: str) -> str:
  try:
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "deepseek/deepseek-r1-0528:free",
            "messages": [{"role": "user", "content": prompt}],
        })
    )

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Ошибка {response.status_code}: {response.text}"

  except Exception as e:
    return f"Ошибка при запросе: {str(e)}"

Сначала найдем все сущности с помощью llm, а после найдем по найденным сущностям их отношения.

In [27]:
def process_text_method_1(chunk_content, api_key):
  ontology = '''
@prefix comcore:  .
@prefix dc:  .
@prefix dcmtype:  .
@prefix dcterms:  .
@prefix owl:  .
@prefix rdf:  .
@prefix rdfs:  .
@prefix xsd:  .

 a owl:Ontology;
     dc:title "Самая общая онтология суперклассов для описания бизнес-процессов сервисов компании" ;
     rdfs:comment """
 Данная онтология описывает суперклассы и их отношения, которые используются при анализе текстовых документов/чанков. Содержит описание сервисов и их вариантов использования - основных классов и составляющих бизнес-процессов: Процессов, Акторов, входных Ресурсов и результирующих продуктов (артефактов), Событий, определяющих цепочку простых Процессов (Актов). """ ;
     rdfs:isDefinedBy ;
     rdfs:seeAlso , ;
     owl:imports , , , ;
     owl:versionIRI ;
     owl:versionInfo "
".

comcore:Process a rdfs:Class;
  rdfs:label "Процесс"@ru;
  rdfs:label "Process"@en;
  rdfs:comment "Процесс, который является подпроцессом Бизнес-процессов";
  dc:description """
Процесс-это продолжительное взаимодействие или работа, выполняемая или инициируемая Акторами (Агентами в роли Акторов) в зависимости от происходящих Событий (сигналов для Акторов, Процессов или других Событий) с использованием Ресурсов с целью создания других Ресурсов и формирования исходов, условий наступления Событий, которые могут стать сигналом для инициации очередного Процесса; """.

comcore:Resource rdfs:subClassOf rdfs:Resource;
  rdfs:label "Ресурс"@ru;
  rdfs:label "Resource"@en;
  rdfs:comment "Это всё, что используется Акторами для реализации Процесса";
  dc:description """
Ресурс - это то, что используется/тратится при реализации процесса или создается в результате процесса. Как правило, за обеспечение Процесса данным Ресурсом отвечает один или несколько Акторов.
Пример: В Процессе подключения Клиента к платежной системе обрабатываются Заявка Клиента на подключение в которой должен обязательно быть его ИНН.
Ресурсы: Заявка Клиента на подключение, ИНН Клиента.""".

comcore:Agent rdfs:subClassOf dcmtype:Agent;
  rdfs:label "Агент"@ru;
  rdfs:label "Agent"@en;
  rdfs:comment "Это сущность, которая активно участвует в Процессе(Бизнес-процессе) и выполняет какую-то роль, реализует определенную функцию";
  dc:description """
Агент - это сущность, которая участвует в Процессе и выполняет какую-то роль, реализует функцию или отвечает за Событие. Агенты взаимодействуют в Процессе. Они инициализируют Подпроцессы, отдельные Акты, результаты которых используют другие Агенты.
Например, для реализации Процесса сопровождения в качестве Агента со стороны компании
может выступать Агент - Сотрудник службы сопровождения, а может выступать Агент - искусственный интеллект или просто какой-то микросервис.
Агент способен получать сообщения и выполнять на их основе какие-то действия, реализовать функцию.
Агент использует для реализации своей функции Ресурсы. Он может быть ответственным за обеспечение Процесса даннным Ресурсом.
Пример: При подключении Клиента к платежной системе задействуются Сотрудник службы безопасности компании, Сотрудник отдела по обслуживанию, сервис мониторинга заполненности ЛК.
Агенты: Сотрудник службы безопасности компании, Сотрудник отдела по обслуживанию, Сервис мониторинга заполненности ЛК.""".

comcore:Event rdfs:subClassOf dcmtype:Event;
  rdfs:label "Событие"@ru;
  rdfs:label "Event"@en;
  rdfs:comment "Событие - стартовое условие запуска следующего Процесса, может быть результатом предыдущего Процесса или инициироваться Агентом (Клиентом, Оператором, ПО)";
  dc:description """
Это описание какого-то исхода в результате реализации Процесса, или События, инициирующего Процесс.
Пример: Проверка MRM задачи может иметь три статуса (Согласовано, Отказано, Нет решения), при отсутствии решения отправляется запрос дополнительных документов.
События: Статус Согласовано, Статус Отказано, Статус Нет решения, Отправка запроса дополнительных документов.""".

comcore:isResultOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasResult;
  rdf:type owl:IrreflexiveProperty;
  rdfs:label "результат"@ru;
  rdfs:label "result"@en;
  rdfs:domain comcore:Resource;
  rdfs:domain comcore:Event;
  rdfs:range comcore:Process;
  rdfs:comment """
Указывает что Событие или Ресурс (субъект) является результатом Процесса (объект).
Пример: Регистрация в форме может закончиться неуспешно если ИНН или Email уже зарегистрирован.
Субъекты: Успешная регистрация, ИНН уже зарегистрирован, Email уже зарегистрирован.""".

comcore:canBeResourceFor rdf:type owl:ObjectProperty;
  rdfs:label "может быть Ресурсом"@ru;
  rdfs:label "может использоваться как Ресурс в"@ru;
  rdfs:label "can be resource for"@en;
  rdfs:label "can be resource in"@en;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:comment """Указывает что Ресурс (субъект) может использоваться в качестве входного Ресурса данного Процесса (объект). Например, ответ на секретный вопрос может использоваться при аутентификации""" .

comcore:isResourceOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasResource;
  rdfs:label "является Ресурсом"@ru;
  rdfs:label "is Resource of"@en;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:comment """Указывает что Ресурс (субъект) является необходимым входным Ресурсом данного Процесса (объект). Например, логин или пароль при аутентификации""" .

comcore:isActorOf rdf:type owl:ObjectProperty;
  owl:inverseOf comcore:hasActor;
  rdfs:label "является Актором"@ru;
  rdfs:label "is Actor of"@en;
  rdfs:comment "Указывает Агента (субъект), который в какой-то роли принимает участие в Процессе (объект)";
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Process.

comcore:Initiates rdfs:subPropertyOf comcore:;
  rdfs:label "инициирует"@ru;
  rdfs:label "initiates"@en;
  rdfs:comment "Указывает что субъект инициирует данный Процесс.";
  rdfs:domain comcore:Event;
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Process.

comcore:isPartOf rdfs:subPropertyOf dcterms:isPartOf;
  owl:inverseOf comcore:hasPart;
  rdf:type owl:IrreflexiveProperty;
  rdf:type owl:TransitiveProperty;
  rdfs:label "является частью"@ru;
  rdfs:label "is part of"@en;
  rdfs:domain comcore:Process;
  rdfs:domain comcore:Resource;
  rdfs:range comcore:Process;
  rdfs:range comcore:Resource;
  rdfs:comment "Указывает, что субъект является частью для данного объекта".

comcore:isResponsibleFor rdfs:subPropertyOf dcterms:contributor;
  rdfs:label "ответственный"@ru;
  rdfs:label "responsible"@en;
  rdfs:comment "Указывает, что Агент (субъект) отвечает за наличие/создание данного Ресурса (объект) или является ответственным за проведение Процесса";
  rdfs:domain comcore:Agent;
  rdfs:range comcore:Resource;
  rdfs:range comcore:Process.

comcore:relation rdfs:subPropertyOf dcterms:relation;
  rdf:type owl:IrreflexiveProperty;
  rdfs:label "связан с"@ru;
  rdfs:label "relation"@en;
  rdfs:comment "Указывает, что одна сущность (субъект) как-то связана с другой сущностью (объект). Отношение используется, если не подходят вышеперечисленные отношения".
'''
     

  prompt_ER = f''' Извлеките из ТЕКСТА все сущности классов, описанных в ОНТОЛОГИИ и ВСЕ отношения между ними;
 сформируй их описание, с использованием RDF-модели в формате Turtle.
ТРЕБОВАНИЯ:
- извлекаются ВСЕ сущности (субъекты и объекты), которые подходят под описание в онтологии;
при этом для каждой извлеченной сущности должны быть созданы минимум три триплета - триплет,
который описывает тип сущности, триплет с именем сущности и отношение между двумя сущностями:
(идентификатор_сущности, rdf:type, ont_name:class_name) ;
(идентификатор_сущности, rdfs:label, "имя сущности") ;
(идентификатор_сущности, идентификатор_предиката, идентификатор_сущности_2) ;
- идентификаторы извлеченных сущностей формируются по правилу ":label", где label - это словоформа, описывающая сущность (метка или имя сущности);
при этом, если словоформа длиннее, чем 40 символов, то она обрезается на 40-м символе; если словоформа состоит из нескольких слов, то она формируется как одно слово,
в котором вместо пробелов, точек подставляется нижнее подчеркивание (например ":оператор_отдела");
- идентификатор_предиката берется из файла описывающего онтологию в соответствии с форматом ont_name:predicate_name;
- сущность должна быть связана с другой сущностью отношением, описанным в ОНТОЛОГИИ:

ТЕКСТ: ```{chunk_content}```
ОНТОЛОГИЯ: ```{ontology}```

ВАЖНО!!!
1) НЕ ДОБАВЛЯТЬ СВОИ ОТНОШЕНИЯ И СУЩНОСТИ
2) НЕ ДОБАВЛЯТЬ НИЧЕГО ЛИШНЕГО
3) ОБРЕЗАТЬ label ДО 40-ГО СИМВОЛА, ЕСЛИ ДЛИНА label > 40

ПРИМЕР ОПИСАНИЯ СУЩНОСТЕЙ:
```
:подключение_к_Системе_МОНЕТА_РУ rdf:type comcore:Process ;
  rdfs:label "Подключение к Системе МОНЕТА.РУ"@ru ;
  comcore:hasPart :проверка_готовности_функционала_сайта_ИМ ;
  comcore:hasPart :регистрация_в_Системе_МОНЕТА_РУ ;
  comcore:hasPart :заполнение_личного_кабинета .

:система_МОНЕТА_РУ rdf:type comcore:Resource ;
  rdfs:label "Система_МОНЕТА.РУ"@ru ;
  comcore:isResourceOf подключение_к_Системе_МОНЕТА_РУ .

:клиент rdf:type comcore:Agent ;
  rdfs:label "Клиент" ;
  comcore:isResposibleFor :личный кабинет ;
  comcore:isActorOf :заполнение личного кабинета .
```
'''
     


  return llm_request(prompt_ER, api_key)

In [23]:
content = """
Подключение к Системе МОНЕТА.РУ - совокупность действий Получателя и/или Торговой площадки, в целях последующего осуществления Перевода с использованием Системы МОНЕТА.РУ:
 - регистрация Получателя в Системе МОНЕТА.РУ;
 - проверка готовности функционала сайта ИМ;
 - установление легитимности продаваемых сайтом товаров и услуг;
 - заполнение личного кабинета (ЛК) Клиента (предоставление в НКО данных о Получателе);
 - создание Бизнес - счета для Получателя в Системе МОНЕТА.РУ.
"""
    
API_KEY = "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"

In [8]:
os.makedirs('./knowledges', exist_ok=True)

In [28]:
text = process_text_method_1(content, API_KEY)
print(text)
with open(f'./knowledges/chunk_test.ttl', 'w') as f:
    f.write(text)

```turtle
@prefix comcore: <http://www.semanticweb.org/comcore#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

:подключение_к_Системе_МОНЕТА_РУ rdf:type comcore:Process ;
  rdfs:label "Подключение к Системе МОНЕТА.РУ"@ru ;
  comcore:hasPart :регистрация_Получателя_в_Систе ,
                  :проверка_готовности_функционала_сайта_ИМ ,
                  :установление_легитимности_продаваемых_са ,
                  :заполнение_личного_кабинета_Клиента ,
                  :создание_Бизнес_счета_для_Получателя_в_С .

:регистрация_Получателя_в_Систе rdf:type comcore:Process ;
  rdfs:label "регистрация Получателя в Системе МОНЕТА.РУ"@ru ;
  comcore:isResourceOf :подключение_к_Системе_МОНЕТА_РУ .

:проверка_готовности_функционала_сайта_ИМ rdf:type comcore:Process ;
  rdfs:label "проверка готовности функционала сайта ИМ"@ru ;
  comcore:isPartOf :подключение_к_Системе_МОНЕТА_РУ .

:установление_легитимности_продаваемых_са

In [29]:
"""
Анализ полученного результата:

@prefix comcore: <http://www.semanticweb.org/comcore#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

:подключение_к_Системе_МОНЕТА_РУ rdf:type comcore:Process ;
  rdfs:label "Подключение к Системе МОНЕТА.РУ"@ru ;
  comcore:hasPart :регистрация_Получателя_в_Систе ,
                  :проверка_готовности_функционала_сайта_ИМ ,
                  :установление_легитимности_продаваемых_са ,
                  :заполнение_личного_кабинета_Клиента ,
                  :создание_Бизнес_счета_для_Получателя_в_С .

:регистрация_Получателя_в_Систе rdf:type comcore:Process ;
  rdfs:label "регистрация Получателя в Системе МОНЕТА.РУ"@ru ;
  comcore:isResourceOf :подключение_к_Системе_МОНЕТА_РУ .

:проверка_готовности_функционала_сайта_ИМ rdf:type comcore:Process ;
  rdfs:label "проверка готовности функционала сайта ИМ"@ru ;
  comcore:isPartOf :подключение_к_Системе_МОНЕТА_РУ .

:установление_легитимности_продаваемых_са rdf:type comcore:Process ;
  rdfs:label "установление легитимности продаваемых сайтом товаров и услуг"@ru ;
  comcore:isPartOf :подключение_к_Системе_МОНЕТА_РУ .

:заполнение_личного_кабинета_Клиента rdf:type comcore:Process ;
  rdfs:label "заполнение личного кабинета (ЛК) Клиента"@ru ;
  comcore:isPartOf :подключение_к_Системе_МОНЕТА_РУ ;
  comcore:isResourceOf :НКО .

:создание_Бизнес_счета_для_Получателя_в_С rdf:type comcore:Process ;
  rdfs:label "создание Бизнес - счета для Получателя в Системе МОНЕТА.РУ"@ru ;
  comcore:isPartOf :подключение_к_Системе_МОНЕТА_РУ .

:Получатель rdf:type comcore:Agent ;
  rdfs:label "Получатель"@ru ;
  comcore:isActorOf :регистрация_Получателя_в_Систе ,
                   :создание_Бизнес_счета_для_Получателя_в_С .

:Торговая_площадка rdf:type comcore:Agent ;
  rdfs:label "Торговая площадка"@ru ;
  comcore:isActorOf :подключение_к_Системе_МОНЕТА_РУ .

:Клиент rdf:type comcore:Agent ;
  rdfs:label "Клиент"@ru ;
  comcore:isActorOf :заполнение_личного_кабинета_Клиента .

:НКО rdf:type comcore:Agent ;
  rdfs:label "НКО"@ru ;
  comcore:isResponsibleFor :личный_кабинет_Клиента .

:сайт_ИМ rdf:type comcore:Resource ;
  rdfs:label "сайт ИМ"@ru ;
  comcore:isResourceOf :проверка_готовности_функционала_сайта_ИМ .

:личный_кабинет_Клиента rdf:type comcore:Resource ;
  rdfs:label "личный кабинет (ЛК) Клиента"@ru ;
  comcore:isResourceOf :заполнение_личного_кабинета_Клиента .

:Бизнес_счет rdf:type comcore:Resource ;
  rdfs:label "Бизнес - счет"@ru ;
  comcore:isResultOf :создание_Бизнес_счета_для_Получателя_в_С .

:Система_МОНЕТА_РУ rdf:type comcore:Resource ;
  rdfs:label "Система МОНЕТА.РУ"@ru ;
  comcore:isResourceOf :подключение_к_Системе_МОНЕТА_РУ .
"""

'\nАнализ полученного результата:\n\n@prefix comcore: <http://www.semanticweb.org/comcore#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n\n:подключение_к_Системе_МОНЕТА_РУ rdf:type comcore:Process ;\n  rdfs:label "Подключение к Системе МОНЕТА.РУ"@ru ;\n  comcore:hasPart :регистрация_Получателя_в_Систе ,\n                  :проверка_готовности_функционала_сайта_ИМ ,\n                  :установление_легитимности_продаваемых_са ,\n                  :заполнение_личного_кабинета_Клиента ,\n                  :создание_Бизнес_счета_для_Получателя_в_С .\n\n:регистрация_Получателя_в_Систе rdf:type comcore:Process ;\n  rdfs:label "регистрация Получателя в Системе МОНЕТА.РУ"@ru ;\n  comcore:isResourceOf :подключение_к_Системе_МОНЕТА_РУ .\n\n:проверка_готовности_функционала_сайта_ИМ rdf:type comcore:Process ;\n  rdfs:label "проверка готовности функционала сайта ИМ"@ru ;\n  comcore:isPartOf :подключение_к_Системе_МОНЕТА_РУ

In [ ]:
TP = 0 # правильно выявленных триплетов
FP = 0 # неправильно описанных триплетов
FN = 0

recall = 100 * TP / (TP + FN)
precision = 100 * TP / (TP + FP)

print(f"- доля извлеченных сущностей= {recall:.2f}%;\n- правильность описания= {precision: .2f}%")

In [30]:
content = """Личный кабинет  -  средство  управления  Кошельком.  Раздел  на  Сайте  Оператора  в  
сети  Интернет,  содержащий  данные  Клиента,  Лицевые  счета  и  иную  информацию,  определяемую  Оператором. 
Доступ в Личный кабинет осуществляется по протоколу HTTPS, с предоставлением возможности настройки двухфакторной аутентификации с 
использованием протокола TOTP."""

In [ ]:
text = process_text_method_1(content, API_KEY)
print(text)
with open(f'./knowledges/chunk_test.ttl', 'w') as f:
    f.write(text)

In [ ]:
"""
Анализ полученного результата
"""

In [ ]:
TP = 0 # правильно выявленных триплетов
FP = 0 # неправильно описанных триплетов
FN = 0
recall = 100 * TP / (TP + FN)
precision = 100 * TP / (TP + FP)

print(f"- доля извлеченных сущностей= {recall:.2f}%;\n- правильность описания= {precision: .2f}%")

In [ ]:
for i in range(count_chunks):
    text = process_text_method_1(read_chunks('./chunks')[i]['content'], API_KEY)
    with open(f'./knowledges/chunk_{i:02d}.ttl', 'w', encoding="windows-1251") as f:
        f.write(text)

: 